In [ ]:
Looking at Psychology Today search results for 07670

In [6]:
from bs4 import BeautifulSoup
import requests, re, time, random
import pandas as pd

In [8]:
headers = {
    'Connection': 'keep-alive',
    'Access-Control-Request-Headers': 'content-type',
    'Accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}
# Grabbing just the results of therapist search in ONE zip code: myzip:
myzip = '11205'
longurl = 'https://therapists.psychologytoday.com/rms/prof_results.php?sid=1508208980.5752_6405&zipcode=07024&rec_next='
zipurl = 'https://therapists.psychologytoday.com/rms/prof_results.php?search=' + myzip
# print(zipurl)
response_zip = requests.get(zipurl, headers=headers).text
soup_zip = BeautifulSoup(response_zip, 'html.parser')
soup_zip

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1.0,maximum-scale=1" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="" name="description"/>
<meta content="noindex,nofollow" name="robots"/> <title>Page Not Found</title>
<link href="https://resources.psychologytoday.com/v3.52.0/css/directoryweb/psychologytoday.en-US.css" rel="stylesheet" type="text/css"/>
<link href="https://resources.psychologytoday.com/v3.52.0/images/favicon-TD.ico" rel="icon" type="image/x-icon"/>
<link href="https://resources.psychologytoday.com/v3.52.0/images/favicon-TD.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="https://resources.psychologytoday.com/v3.52.0/images/apple-touch-icon-TD.png" rel="apple-touch-icon"/>
<script src="https://resources.psychologytoday.com/v3.52.0/js/jquery/jquery-1.12.4.min.js" type="text/javascript"></script>
<script src="https://resources.psychologytoday.com/v3.52.0/j

In [3]:
# Manually constructing the list of urls for ALL available search result pages in that tip - 20 therapists per page:
pages_url_list = []
i = 1
while True:
    myurl = longurl+str(i)
    response = requests.get(myurl, headers=headers).text
    soup = BeautifulSoup(response, 'html.parser')
    alert = soup.find('div', attrs={'class': 'alert-alert'})
    if alert is None:
        i = i + 20
        pages_url_list.append(myurl)
        # time.sleep(random.randint(1,2))
    else:
        break

In [4]:
print(pages_url_list)  # This list contains URLs of all pages that contain therapists found for this zip
print(len(pages_url_list))

['https://therapists.psychologytoday.com/rms/prof_results.php?sid=1508208980.5752_6405&zipcode=07024&rec_next=1', 'https://therapists.psychologytoday.com/rms/prof_results.php?sid=1508208980.5752_6405&zipcode=07024&rec_next=21', 'https://therapists.psychologytoday.com/rms/prof_results.php?sid=1508208980.5752_6405&zipcode=07024&rec_next=41', 'https://therapists.psychologytoday.com/rms/prof_results.php?sid=1508208980.5752_6405&zipcode=07024&rec_next=61']
4


In [5]:
# Loop through all urls found above (for pages of therapist search for that zip) and grab the urls of each and every therapist:
therapist_urls = []
for url in pages_url_list:
    response = requests.get(url, headers = headers).text
    soup = BeautifulSoup(response, 'html.parser')
    therapists = soup.find_all('a', {'class': "result-name"})
    for i in range(len(therapists)):
        therapist_urls.append(therapists[i]["href"])
        # time.sleep(random.randint(1,2))

In [6]:
print(len(therapist_urls))

65


In [57]:
# For a single therapist, parse his/her page:
ter = 0
# response_one = requests.get(therapist_urls[ter], headers=headers).text
# temp = therapist_urls[ter]
# temp = 'https://therapists.psychologytoday.com/rms/prof_detail.php?profid=35956&rec_next=1&ref=16&sid=1508110622.8452_29222&zipcode=11205&tr=ResultsName'
temp = 'https://therapists.psychologytoday.com/rms/prof_detail.php?profid=147000&rec_next=1&ref=1&sid=1508249435.4562_29222&zipcode=10012&tr=ResultsName'
response_one = requests.get(temp, headers=headers).text
one_therap = BeautifulSoup(response_one, 'html.parser')
therapist_urls[27]
temp

NameError: name 'therapist_urls' is not defined

In [5]:
# Grab qualifications of one therapist
qualif_div = one_therap.find('div', {'class': "profile-qualifications"})
allqualif = qualif_div.find_all('li')   # they are saved in 'li'
# allqualif

In [6]:
# Collecting Years in Practice:

years_check = [i.text.split('\n')[2].strip() for i in allqualif if "Years in Practice" in i.text]
if len(years_check) > 0:
    years_in_practice = years_check[0].replace(' Years', '')
else:
    years_in_practice = ''
years_in_practice

'20+'

In [12]:
# Collecting School:

school_check = [i.text.split('\n')[2].strip() for i in allqualif if "School" in i.text]
if len(school_check) > 0:
    school = school_check[0]
    school = re.sub(r'\([^)]*\)', '', school).strip()
    school = school.replace(', ', '_')
    school = school.replace(',', '_')
    school = school.replace('/ ', '_')
    school = school.replace('/', '_')
    school = school.replace(' - ', '_')
    school = school.replace(' ', '_').lower()
else:
    school = ''

In [13]:
school

'hunter_college_grad_school_of_social_work'

In [14]:
# Collecting Year Graduated:

yeargrad_check = [i.text.split('\n')[2].strip() for i in allqualif if "Year Graduated" in i.text]
if len(yeargrad_check) > 0:
    yeargrad = int(yeargrad_check[0])
else:
    yeargrad = ''

In [15]:
yeargrad

1978

In [16]:
# Collecting Board Certification:

board_check = [i.text.split('\n')[2].strip() for i in allqualif if "Board Certification" in i.text]
if len(board_check) > 0:
    board_certif = 1
else:
    board_certif = 0

In [17]:
board_certif

0

In [18]:
# Collecting License number & State:

license_check = [i.text.split('\n')[2].strip() for i in allqualif if "License No. and State" in i.text]
if len(license_check) > 0:
    # license = license_check[0]
    license = license_check[0].split()[0]
    license_state = " ".join(license_check[0].split()[1:])
else:
    license = ''
    license_state = ''

In [19]:
print(license)
print(license_state)

R022106-1
New York


In [20]:
# Grabbing all titles of one therapist
titles = one_therap.find('div', {'class': "profile-title"}).text.strip()
titles = titles.replace("\n", "")
titles = titles.replace('/', '_').strip()
titles = titles.replace(', ', ',')
titles = titles.replace(' ', '_')
titles = titles.replace(',', ', ')

In [21]:
titles

'Clinical_Social_Work_Therapist, LCSW'

In [22]:
finances = one_therap.find('div', {'class': "profile-finances"})
finances == None
# Grabbing financial info:
finances = one_therap.find('div', {'class': "profile-finances"})
# print(finances)

In [23]:
finances == None

False

In [24]:
# Grabbing several financial metrics:
lis = finances.find_all('li')

# Grab cost per session:
persession_check = [i.text.split(': ')[1].strip() for i in lis if "Avg Cost (per session)" in i.text]
if len(persession_check) > 0:
    per_session = persession_check[0].replace('$', '')
    per_session = per_session.replace(' ', '')
else:
    per_session = ''
per_session

'100-170'

In [25]:
# print(lis)
# for i in lis:
#     print(i.text)
sliding_check = [i.text for i in lis if 'Sliding Scale' in i.text]
if sliding_check != None:
    sliding_check = " ".join(sliding_check)
print(sliding_check)
'Sliding Scale: Yes' in sliding_check


Sliding Scale: Yes
            


True

In [26]:
sliding_check = [i.text for i in lis if 'Sliding Scale' in i.text]
if sliding_check != None:
    sliding_check = " ".join(sliding_check)
    if 'Sliding Scale: Yes' in sliding_check:
        sliding = 1
    else:
        sliding = 0
else:
    sliding = 0
sliding

1

In [27]:
sliding_check = [i.text.split(': ')[1].strip() for i in lis if "Sliding Scale" in i.text]

In [28]:
# Grab sliding scale (1 = yes, 0 = no):
sliding_check = [i.text.split(': ')[1].strip() for i in lis if "Sliding Scale" in i.text]
if len(sliding_check) > 0:
    sliding = 1
else:
    sliding = 0

In [29]:
# Grabbing financial info:
finances = one_therap.find('div', {'class': "profile-finances"})
print(finances)

if finances == None:
    per_session = ''
    sliding = ''
    insurance_yes = ''
else:
    # Grabbing several financial metrics:
    lis = finances.find_all('li')

    # Grab cost per session:
    persession_check = [i.text.split(': ')[1].strip() for i in lis if "Avg Cost (per session)" in i.text]
    if len(persession_check) > 0:
        per_session = persession_check[0].replace('$', '')
        per_session = per_session.replace(' ', '')
    else:
        per_session = ''

    # Grab sliding scale (1 = yes, 0 = no):
    sliding_check = [i.text.split(': ')[1].strip() for i in lis if "Sliding Scale" in i.text]
    if len(sliding_check) > 0:
        sliding = 1
    else:
        sliding = 0
    
    # Grab Accepts Insurance - yes or no:
    insur_check = [i.text.split(': ')[1].strip() for i in lis if "Accepts Insurance" in i.text]
    if len(insur_check) > 0:
        insurance_yes = 1   # insur_check[0]
    else:
        insurance_yes = 0    # ''


<div class="profile-finances">
<h2>Finances</h2>
<ul>
<li>
<strong>Avg Cost (per session):</strong> $100 - $170
            </li>
<li>
<strong>Sliding Scale:</strong> Yes
            </li>
<li>
<strong>Accepts Insurance:</strong> Yes
            </li>
<li>
<strong>Accepted Payment Methods: </strong>
                                    Cash,                                     Check                            </li>
</ul>
<br/><strong>Accepted Insurance Plans</strong>
<div class="spec-list">
<div class="row">
<div class="col-xs-12 col-sm-6 col-md-6 col-lg-6 col-tight-right">
<ul>
<li>BlueCross and BlueShield</li>
<li>EmblemHealth</li>
<li>Empire BlueCross</li>
<li>GHI</li>
<li>MagnaCare</li>
</ul>
</div>
<div class="col-xs-12 col-sm-6 col-md-6 col-lg-6 col-tight-right">
<ul>
<li>Optum</li>
<li>Oxford</li>
<li>United Health Care</li>
<li>ValueOptions</li>
<li>Out of Network</li>
</ul>
</div>
</div>
</div>
<div class="profile-verify-ins">
            Verify your health insurance coverage w

In [30]:
per_session

'100-170'

In [31]:
sliding

1

In [32]:
insurance_yes

1

In [33]:
payment_check = [i.text for i in lis if "Accepted Payment Methods" in i.text]
payment_check

['\nAccepted Payment Methods: \n                                    Cash,                                     Check                            ']

In [34]:
# Grab Methods of Payment:
# payment_check = [i.text for i in lis if "Accepted Payment Methods" in i.text]
payment_check = [i.text.split('\n')[2].strip().split(',') for i in lis if "Accepted Payment Methods" in i.text]
if len(payment_check) == 0:
    payment_methods = ''
else:
    payment_methods = [item for sublist in payment_check for item in sublist]
    # print(payment_check)
    # print(len(payment_check))
    for index, item in enumerate(payment_methods):
        out = item.strip().lower()
        out = out.replace(" ", "_")
        payment_methods[index] = out
payment_methods = ', '.join(payment_methods)
# payment_methods

In [35]:
payment_methods

'cash, check'

In [71]:
long = one_therap.find_all('div', {'class': "col-xs-12 col-sm-6 col-md-6 col-lg-6 col-tight-right"})
# long

In [36]:
# Grab Accepted Insurance Plans:

long = one_therap.find_all('div', {'class': "col-xs-12 col-sm-6 col-md-6 col-lg-6 col-tight-right"})
if len(long) > 0:
    result = []
    for i in long:
        result.extend(i.find_all('li'))
    plans = []
    for i in range(len(result)):
        out = result[i].text
        out = re.sub(r'\([^)]*\)', '', out)
        out = out.replace(' ', '_')
        out = out.replace('-', '_')
        plans.append(out)   # plans.append(result[i].text)
    plans = ', '.join(plans).lower()
else:
    plans = ''
plans

'bluecross_and_blueshield, emblemhealth, empire_bluecross, ghi, magnacare, optum, oxford, united_health_care, valueoptions, out_of_network'

In [38]:
spec_check = one_therap.find_all('li', {'class': "highlight"})
spec_check

[]

In [40]:
len(spec_check)

0

In [46]:
# Grab specialties:

spec_check = one_therap.find_all('li', {'class': "highlight"})
if len(spec_check) == 0:
    specialties = ''
else:
    specialties = []
    for i in range(len(spec_check)):
        out = spec_check[i].text.replace(', ', '_')
        out = out.replace(' ', '_').lower()
        specialties.append(out)
    specialties = ", ".join(specialties)

In [47]:
specialties

''

In [48]:
# Grab issues:
import re
issues_check = one_therap.find_all('div', {'class': 'col-xs-12 col-sm-12 col-md-6 col-lg-6'})
if len(issues_check) == 0:
    issues = ''
else:
    result = []
    for i in issues_check:
        result.extend(i.find_all('li'))

    issues = []
    for i in range(len(result)):
        out = result[i].text.strip()
        out = re.sub(r'\([^)]*\)', '', out)
        out = re.sub(r' $', '', out)
        out = out.replace(', ', '_')
        out = out.replace('-', '_')
        out = out.replace('\'', '')
        out = out.replace(' ', '_').lower()
        issues.append(out)
    issues = ", ".join(issues)
issues

''

In [49]:
# Grab preferred ages:
 
# In rare cases when no specialties were selected, we need to grab the 3rd, not the 4th element:
if specialties == '':
    age_check = one_therap.find_all('div', {'class': 'col-xs-12 col-sm-12 col-md-12 col-lg-12'})[2]
else:
    age_check = one_therap.find_all('div', {'class': 'col-xs-12 col-sm-12 col-md-12 col-lg-12'})[3]
len(age_check)
age_check

age_lis = age_check.find_all('li')

ages = []
for i in age_lis:
    ages.append(i.text)

# Creating values for age categories:

# Age - toddlers:
if "Toddlers / Preschoolers (0 to 6)" in ages:
    age_toddlers = 1
else:
    age_toddlers = 0

# Age - children:
if "Children (6 to 10)" in ages:
    age_children = 1
else:
    age_children = 0
    
# Age - preteens:
if "Preteens / Tweens (11 to 13)" in ages:
    age_preteens = 1
else:
    age_preteens = 0

# Age - teens:
if "Adolescents / Teenagers (14 to 19)" in ages:
    age_teens = 1
else:
    age_teens = 0

# Age - adults:
if "Adults" in ages:
    age_adults = 1
else:
    age_adults = 0

# Age - elders:
if "Elders (65+)" in ages:
    age_elders = 1
else:
    age_elders = 0
agesum = age_toddlers+age_children+age_preteens+age_teens+age_adults+age_elders

In [50]:
print(age_toddlers)
print(age_children)
print(age_preteens)
print(age_teens)
print(age_adults)
print(age_elders)

0
0
0
0
0
0


In [51]:
agesum = age_toddlers+age_children+age_preteens+age_teens+age_adults+age_elders
agesum

0

In [52]:
# Grab subcategories
if specialties == '' and agesum == 0:
    subcat_check = one_therap.find_all('div', {'class': 'col-xs-12 col-sm-12 col-md-12 col-lg-12'})[2]
elif specialties == '' and agesum > 0:
    subcat_check = one_therap.find_all('div', {'class': 'col-xs-12 col-sm-12 col-md-12 col-lg-12'})[3]
elif specialties != '' and agesum == 0:
    subcat_check = one_therap.find_all('div', {'class': 'col-xs-12 col-sm-12 col-md-12 col-lg-12'})[3]
else:
    subcat_check = one_therap.find_all('div', {'class': 'col-xs-12 col-sm-12 col-md-12 col-lg-12'})[4]
len(subcat_check)
subcat_check

subcat_lis = subcat_check.find_all('li')

subcats = []
for i in subcat_lis:
    subcats.append(i.text)

if "Aviation Professionals" in subcats:  # Subcategories - Aviation:
    sub_pilots = 1
else:
    sub_pilots = 0

if "Bisexual Clients" in subcats:  # Subcategories - HIV:
    sub_bisexuals = 1
else:
    sub_bisexuals = 0

if "Cancer" in subcats:      # Subcategories - Cancer:
    sub_cancer = 1
else:
    sub_cancer = 0

if "Gay Clients" in subcats:  # Subcategories - Gays:
    sub_gays = 1
else:
    sub_gays = 0

if "HIV / AIDS Clients" in subcats:  # Subcategories - HIV:
    sub_hiv = 1
else:
    sub_hiv = 0

if "Heterosexual Clients" in subcats:  # Subcategories - Heterosexual:
    sub_heteros = 1
else:
    sub_heteros = 0

if "Lesbian Clients" in subcats:   # Subcategories - Lesbian:
    sub_lesbians = 1
else:
    sub_lesbians = 0

if "Transgender Clients" in subcats:  # Subcategories - Transgender:
    sub_transgender = 1
else:
    sub_transgender = 0

if "Veterans" in subcats:   # Subcategories - Veterans:
    sub_veterans = 1
else:
    sub_veterans = 0

subsum = sub_pilots+sub_bisexuals+sub_cancer+sub_gays+sub_hiv+sub_heteros+sub_lesbians+sub_transgender+sub_veterans
print(subsum)

0


In [53]:
print(sub_pilots)
print(sub_bisexuals)
print(sub_cancer)
print(sub_gays)
print(sub_hiv)
print('\n')
print(sub_heteros)
print(sub_lesbians)
print(sub_transgender)
print(sub_veterans)

0
0
0
0
0


0
0
0
0


In [56]:
approach_check = one_therap.find_all('div', {'class': 'col-xs-12 col-sm-12 col-md-12 col-lg-12'})
approach_check[0]

<div class="col-xs-12 col-sm-12 col-md-12 col-lg-12">
<h3>Zips:</h3>
<ul>
<li>
<a class="textNoLink" href="https://therapists.psychologytoday.com/rms/zip/10012.html" title="Therapists in 10012">10012</a>
</li>
<li>
<a class="textNoLink" href="https://therapists.psychologytoday.com/rms/zip/10015.html" title="Therapists in 10015">10015</a>
</li>
<li>
<a class="textNoLink" href="https://therapists.psychologytoday.com/rms/zip/10016.html" title="Therapists in 10016 (Kips Bay, Murray Hill)">10016 (Kips Bay, Murray Hill)</a>
</li>
</ul>
</div>

In [76]:
approach_check = one_therap.find_all('h2')
asalist = []
count = 1
for i in approach_check:
    try:
        out = i.text
    except:
        print("some error")
    asalist.append(out)
print(asalist)
'Treatment Approach' in asalist

['\nPsychiatrist, \nMD\n', '\nPsychiatrist, \nMD\n', 'Location', 'Nearby Areas', 'Qualifications', 'Additional Credentials', 'Location', 'Nearby Areas', 'Specialties', 'Client Focus', 'Nearby Areas', '\nPsychiatrist, \nMD\n']


False

In [43]:
# Grab treatment approaches:
age_and_sub_sum = agesum + subsum
print(age_and_sub_sum)
approach_check = one_therap.find_all('div', {'class': 'col-xs-12 col-sm-12 col-md-12 col-lg-12'})
if specialties == '' and age_and_sub_sum == 0:           # if all 3 are not there
    approach_check = approach_check[2]
elif specialties != '' and age_and_sub_sum == 0:         # if only 2 out of 3 are not there
    approach_check = approach_check[3]
elif specialties == '' and agesum == 0 and subsum > 0:
    approach_check = approach_check[3]
elif specialties == '' and agesum > 0 and subsum == 0:
    approach_check = approach_check[3]
elif specialties != '' and agesum == 0  and subsum > 0:  # if only 1 out of 3 is not there
    approach_check = approach_check[4]
elif specialties != '' and agesum > 0  and subsum == 0:
    approach_check = approach_check[4]
elif specialties == '' and agesum > 0  and subsum > 0:
    approach_check = approach_check[4]
else:
    approach_check = approach_check[5]

approach_lis = approach_check.find_all('li')

approaches = []
for i in approach_lis:
    approaches.append(i.text)

for i in range(len(approaches)):
    out = approaches[i]
    out = out.replace(' / ', '_')
    out = re.sub(r'\([^)]*\)', '', out)
    out = re.sub(r' $', '', out)
    out = out.strip().replace('-', '_')
    out = out.replace('/', '')
    out = out.replace('\'', '')
    out = out.replace(' ', '_').lower()
    out = 'approach_' + out
    approaches[i] = out
approaches = ', '.join(approaches)
print(approaches)

1
approach_eclectic


In [75]:
# For a single therapist, parse his/her page:
ter = 1
# temp = therapist_urls[ter]
# temp = 'https://therapists.psychologytoday.com/rms/prof_detail.php?profid=239688&ref=5&sid=1508208616.6699_11157&zipcode=07670&tr=ResultsName'
temp = 'https://therapists.psychologytoday.com/rms/prof_detail.php?profid=143075&ref=7&sid=1508208616.6699_11157&zipcode=07670&tr=ResultsPhoto'
temp = 'https://therapists.psychologytoday.com/rms/prof_detail.php?profid=57208&ref=12&sid=1508208616.6699_11157&zipcode=07670&tr=ResultsPhoto'
temp = 'https://therapists.psychologytoday.com/rms/prof_detail.php?profid=110966&ref=14&sid=1508208616.6699_11157&zipcode=07670&tr=ResultsName'
temp = 'https://therapists.psychologytoday.com/rms/prof_detail.php?profid=264862&ref=2&sid=1508208616.6699_11157&zipcode=07670&tr=ResultsPhoto'
temp = 'https://therapists.psychologytoday.com/rms/prof_detail.php?profid=268880&ref=16&sid=1508208616.6699_11157&zipcode=07670&tr=ResultsPhoto'
temp = 'https://therapists.psychologytoday.com/rms/prof_detail.php?profid=328027&rec_next=21&ref=9&sid=1508208616.6699_11157&zipcode=07670&tr=ResultsPhoto'
temp = 'https://therapists.psychologytoday.com/rms/prof_detail.php?profid=132006&search=10012&ref=1&sid=1508260598.8676_22538&zipcode=10012&tr=ResultsName'
temp = 'https://therapists.psychologytoday.com/rms/prof_detail.php?profid=182958&rec_next=21&ref=8&sid=1508249435.4562_29222&zipcode=10012&tr=ResultsName'
response_one = requests.get(temp, headers=headers).text
one_therap = BeautifulSoup(response_one, 'html.parser')
# therapist_urls[ter]
temp


'https://therapists.psychologytoday.com/rms/prof_detail.php?profid=182958&rec_next=21&ref=8&sid=1508249435.4562_29222&zipcode=10012&tr=ResultsName'

In [54]:
# grab additional languages
spec_subcat = one_therap.find_all('div', {'class': "spec-subcat"})
spec_subcat

[]

In [46]:
# grab additional languages
spec_subcat = one_therap.find_all('div', {'class': "spec-subcat"})
list_of_children = [None] * len(spec_subcat)
for i in range(len(spec_subcat)):
    list_of_children[i] = spec_subcat[i].findChildren()

In [47]:
# grab additional languages
spec_subcat = one_therap.find_all('div', {'class': "spec-subcat"})
list_of_children = [None] * len(spec_subcat)
for i in range(len(spec_subcat)):
    list_of_children[i] = spec_subcat[i].findChildren()
whatsthere = []
for i in range(len(list_of_children)):
    for z in range(len(list_of_children[i])):
        for t in [text for text in list_of_children[i][z].stripped_strings]:
            whatsthere.append(t)
for i in range(len(whatsthere)):
    whatsthere[i] = whatsthere[i].replace(',', '')


In [48]:
# Grab ethnicities, languages, religious orientation:
spec_subcat = one_therap.find_all('div', {'class': "spec-subcat"})
list_of_children = [None] * len(spec_subcat)
for i in range(len(spec_subcat)):
    list_of_children[i] = spec_subcat[i].findChildren()
whatsthere = []
for i in range(len(list_of_children)):
    for z in range(len(list_of_children[i])):
        for t in [text for text in list_of_children[i][z].stripped_strings]:
            whatsthere.append(t)
for i in range(len(whatsthere)):
    whatsthere[i] = whatsthere[i].replace(',', '')

# Finding indexes for Ethnicity, Languages, and Religious Orientation:
if one_therap.find_all("strong", string="Ethnicity:") == []:
    ethnicities = ''
    index_ethnic = ''
else:
    index_ethnic = whatsthere.index("Ethnicity:")

if one_therap.find_all("strong", string="Alternative Languages:") == []:
    languages = ''
    index_languages = ''
else:
    index_languages = whatsthere.index("Alternative Languages:")
index_languages
if one_therap.find_all("strong", string="Religious Orientation:") == []:
    religion = ''
    index_religion = ''
else:
    index_religion = whatsthere.index("Religious Orientation:")

# Grabbing applicable info - depending on what's there and what's not:
if index_ethnic != '' and index_languages != '' and index_religion != '':  # if all 3 are present
    ethnicities = whatsthere[(index_ethnic+1):index_languages]
    languages = whatsthere[(index_languages+1):index_religion]
    religion = whatsthere[(index_religion+1):len(whatsthere)]

if index_ethnic == '' and index_languages != '' and index_religion != '':   # if only languages and religion present
    languages = whatsthere[(index_languages+1):index_religion]
    religion = whatsthere[(index_religion+1):len(whatsthere)]

if index_ethnic != '' and index_languages == '' and index_religion != '':   # if only ethnicities and religion present
    ethnicities = whatsthere[(index_ethnicities+1):index_religion]
    religion = whatsthere[(index_religion+1):len(whatsthere)]

if index_ethnic != '' and index_languages != '' and index_religion == '':   # if only ethnicities and languages present
    ethnicities = whatsthere[(index_ethnicities+1):index_languages]
    languages = whatsthere[(index_languages+1):len(whatsthere)]

if index_ethnic != '' and index_languages == '' and index_religion != '':   # if only ethnicities and religion present
    ethnicities = whatsthere[(index_ethnicities+1):index_religion]
    religion = whatsthere[(index_religion+1):len(whatsthere)]

if index_ethnic != '' and index_languages == '' and index_religion == '':   # if only ethnicities present
    ethnicities = whatsthere[(index_ethnicities+1):len(whatsthere)]

if index_ethnic == '' and index_languages != '' and index_religion == '':   # if only languages present
    languages = whatsthere[(index_languages+1):len(whatsthere)]

if index_ethnic == '' and index_languages == '' and index_religion != '':   # if only religion present
    religion = whatsthere[(index_religion+1):len(whatsthere)]

ethnicities = ", ".join(ethnicities)
languages = ", ".join(languages)
religion = ", ".join(religion)

In [49]:
print(ethnicities)
print(languages)
print(religion)


Hebrew
Jewish


In [197]:
i = list_of_children[0]
print(i)
z = i[0]
print(z)
mytext = [text for text in z.stripped_strings]
print(mytext)

[<strong>Ethnicity:</strong>, <span>African-American, </span>, <span>Hispanic and Latino</span>]
<strong>Ethnicity:</strong>
['Ethnicity:']


In [201]:
strongs = []
for i in list_of_children:
    print(i)
    print('='*10)
    for z in i:
        print(z)
        print('*'*5)
        mytext = [text for text in z.stripped.strings]
        # print(words)

[<strong>Ethnicity:</strong>, <span>African-American, </span>, <span>Hispanic and Latino</span>]
<strong>Ethnicity:</strong>
*****


AttributeError: 'NoneType' object has no attribute 'strings'

In [122]:
list_of_strongs = [None] * len(list_of_children)
for i in range(0, len(list_of_children)):
    list_of_strongs[i] = list_of_children[i].find('strong')

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [121]:
len(list_of_children[0])

3

In [114]:
type(list_of_children[0])

bs4.element.ResultSet

In [90]:
out = []
for i in range(0, len(spec_subcat)):
    out[i] = spec_subcat[i].find('span')

IndexError: list assignment index out of range

In [95]:
spec_subcat[1].find('span').text

'Spanish'

In [57]:
# grab modality
all_lis = one_therap.find_all('li')
all_lis[0]
# texts = []
for i in all_lis:
    print(i.tex

SyntaxError: unexpected EOF while parsing (<ipython-input-57-bb8d2efcdf57>, line 6)

In [1131]:
individuals = [i for i in all_lis.t]


[<li><a href="https://therapists.psychologytoday.com/rms/?tr=Header_Menu2" title="Find Therapists">Therapists</a></li>,
 <li><a href="https://psychiatrists.psychologytoday.com/rms/?tr=Header_Menu2" title="Find Psychiatrists">Psychiatrists</a></li>,
 <li><a href="https://groups.psychologytoday.com/rms/?tr=Header_Menu2" title="Find Support Groups">Support Groups</a></li>,
 <li><a href="https://treatment.psychologytoday.com/rms/?tr=Header_Menu2" title="Find Treatment Centers">Treatment Centers</a></li>,
 <li><br/><a href="https://member.psychologytoday.com/rms-sec/">Log In</a></li>,
 <li><a href="https://member.psychologytoday.com/therapist/?src=RMSLOG">Sign Up</a></li>,
 <li>
 <a class="active" href="https://therapists.psychologytoday.com/rms/prof_detail.php?profid=94804&amp;p=1&amp;sid=1507757611.7036_14284&amp;zipcode=07670&amp;rec_next=1&amp;ref=4" title="About">About</a>
 </li>,
 <li>
 <a href="https://therapists.psychologytoday.com/rms/prof_detail.php?profid=94804&amp;p=5&amp;sid=15